<a href="https://colab.research.google.com/github/vkjadon/openai/blob/main/05oai_evaluator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

We can load the api key from `.env` file by importing `load_dotenv` method of `dotenv` package. We can install `dotenv` package as

> `pip install python-dotenv`

> `from dotenv import load_dotenv, find_dotenv`

`find_dotenv()` : This function searches up the directory tree from your current working directory until it locates the .env file.

`load_dotenv()` : This function reads the variables from the found .env file and loads them into your Python process's environment variables (specifically, os.environ).

In [ ]:
from openai import OpenAI
from dotenv import load_dotenv, find_dotenv

load_dotenv(find_dotenv())

client = OpenAI()

In case you are running the code on Google Colab, add your "OPENAI_API_KEY" key in the `Secrets` of the colab environment and call instantiate the `OpenAI` object as below

In [5]:
!pip install -q faiss-cpu pypdf tiktoken

In [6]:
from openai import OpenAI
from google.colab import userdata
client = OpenAI(api_key=userdata.get('OPENAI_API_KEY'))

In [7]:
from google.colab import files

uploaded = files.upload()
pdf_path = list(uploaded.keys())[0]

Saving Chap-07-knucckle and cotter.pdf to Chap-07-knucckle and cotter.pdf


In [8]:
from pypdf import PdfReader

reader = PdfReader(pdf_path)

text = ""
for page in reader.pages:
    text += page.extract_text()

print(text[:1000])  # preview


/G43/G6F/G74/G74/G65/G72/G20/G6A/G6F/G69/G6E/G74/G20/G69/G73/G20/G75/G73/G65/G64/G20/G77/G68/G65/G6E/G20/G74/G68/G65/G20/G6D/G65/G6D/G62/G65/G72/G73/G20/G61/G72/G65/G20/G73/G75/G62/G6A/G65/G63/G74/G65/G64/G20/G74/G6F/G20/G61/G78/G69/G61/G6C/G20/G74/G65/G6E/G73/G69/G6C/G65/G20/G6F/G72/G20/G63/G6F/G6D/G70/G72/G65/G73/G73/G69/G76/G65/G20/G6C/G6F/G61/G64/G73/G2C/G20/G65/G61/G73/G79
/G61/G73/G73/G65/G6D/G62/G6C/G79/G20/G61/G6E/G64/G20/G64/G69/G73/G61/G73/G73/G65/G6D/G62/G6C/G79/G20/G69/G73/G20/G72/G65/G71/G75/G69/G72/G65/G64/G2C/G20/G6E/G6F/G20/G72/G65/G6C/G61/G74/G69/G76/G65/G20/G6D/G6F/G74/G69/G6F/G6E/G20/G62/G65/G74/G77/G65/G65/G6E/G20/G74/G68/G65/G20/G74/G77/G6F/G20/G66/G61/G73/G74/G65/G6E/G65/G64/G20/G6D/G65/G6D/G62/G65/G72/G73
/G28/G72/G6F/G64/G73/G29/G20/G69/G73/G20/G64/G65/G73/G69/G72/G65/G64/G20/G61/G6E/G64/G20/G74/G68/G65/G20/G61/G78/G65/G73/G20/G6F/G66/G20/G74/G77/G6F/G20/G72/G6F/G64/G73/G20/G61/G72/G65/G20/G63/G6F/G6C/G6C/G69/G6E/G65/G61/G72/G2E/G20/G54/G68/G65/G20/G63/G6F/G74/G

In [9]:
def chunk_text(text, chunk_size=800, overlap=100):
    chunks = []
    start = 0
    while start < len(text):
        end = start + chunk_size
        chunks.append(text[start:end])
        start = end - overlap
    return chunks

In [10]:
chunks = chunk_text(text)
print("Total chunks:", len(chunks))

Total chunks: 142


In [11]:
import numpy as np

In [12]:
def get_embeddings(texts):
    response = client.embeddings.create(
        model="text-embedding-3-small",
        input=texts
    )
    return np.array([e.embedding for e in response.data])

In [13]:
chunk_embeddings = get_embeddings(chunks)
print(chunk_embeddings.shape)

(142, 1536)


FAISS (Facebook AI Similarity Search) is an open-source library for efficient similarity search and clustering of dense, high-dimensional vectors. It is used to quickly find items in massive datasets that are "similar" to a given query item, solving performance bottlenecks that traditional databases face when dealing with unstructured data like images or text embeddings.

In [14]:
import faiss

dimension = chunk_embeddings.shape[1]
index = faiss.IndexFlatL2(dimension)
index.add(chunk_embeddings)

print("FAISS index size:", index.ntotal)

FAISS index size: 142


In [15]:
def retrieve_context(query, k=4):
    query_embedding = get_embeddings([query])
    distances, indices = index.search(query_embedding, k)
    return "\n\n".join([chunks[i] for i in indices[0]])

In [16]:
def build_context(query):
    retrieved = retrieve_context(query)
    return "\n\n".join(retrieved)

In [17]:
asked_questions = []
def generate_unique_question(topic, difficulty="medium"):
    context = build_context(topic)

    previous_questions = "\n".join(asked_questions) if asked_questions else "None"

    response = client.responses.create(
        model="gpt-4.1-mini",
        input=[
            {
                "role": "system",
                "content": "You are a university examiner."
            },
            {
                "role": "developer",
                "content": (
                    "Generate ONE new exam question "
                    "that has NOT been asked before in this session. "
                    "Do not repeat or rephrase previous questions."
                )
            },
            {
                "role": "user",
                "content": f"""
Context:
{context}

Previously asked questions:
{previous_questions}

Topic:
{topic}
"""
            }
        ]
    )

    question = response.output_text.strip()
    asked_questions.append(question)

    return question


In [20]:
question = generate_unique_question("Cotter Joint")
print("QUESTION:", question)

QUESTION: Using the dimensional symbols and proportions described in the context, derive an expression for the shear area of the cotter in terms of the rod diameter (d), cotter width (b), and cotter thickness (t). Discuss how variations in each parameter would affect the strength and durability of the cotter joint under shear loading.


In [21]:
def evaluate_answer(question, student_answer, max_marks=10):
    context = build_context(question)

    response = client.responses.create(
        model="gpt-4.1-mini",
        input=[
            {
                "role": "system",
                "content": "You are a strict and fair examiner."
            },
            {
                "role": "developer",
                "content": (
                    "Evaluate the student answer using ONLY the provided context. "
                    "Do NOT use external knowledge.\n\n"
                    "Follow this rubric:\n"
                    "1. Concept correctness\n"
                    "2. Coverage of key points\n"
                    "3. Technical accuracy\n"
                    "4. Clarity of explanation\n\n"
                    f"Assign marks out of {max_marks}. "
                    "Return output in JSON with keys: "
                    "score, strengths, missing_points, feedback."
                )
            },
            {
                "role": "user",
                "content": f"""
Context:
{context}

Question:
{question}

Student Answer:
{student_answer}
"""
            }
        ]
    )

    return response.output_text


In [22]:
student_answer = input("Enter student answer:\n")

Enter student answer:
Shear Area = 2 * pi * t * b


In [23]:
evaluation = evaluate_answer(question, student_answer)
print("EVALUATION RESULT:\n", evaluation)


EVALUATION RESULT:
 ```json
{
  "score": 2,
  "strengths": [
    "Student attempts to provide a formula for shear area, indicating awareness of relevant parameters (t and b)."
  ],
  "missing_points": [
    "The expression for shear area should properly include the rod diameter (d), as the cotter shear planes relate to d and the cotter dimensions.",
    "The coefficient '2 * pi' is incorrect; shear area here is related to planar areas, not circular or cylindrical surface area.",
    "Lack of explanation about how shear area is derived geometrically from the cotter joint dimensions in the context.",
    "No discussion on how variations in rod diameter (d), cotter width (b), and thickness (t) affect strength and durability.",
    "No explanation tying the dimensions to strength and durability under shear loading."
  ],
  "feedback": "The provided formula for shear area '2 * pi * t * b' is incorrect based on the context's dimensional symbols and proportions. The shear area of the cotter i